In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import ipynb.fs.full.features as features
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import make_classification
import ipynb.fs.full.utils as utils


In [2]:
df = pd.read_csv("./data/train.csv")

df = features.llenar_nulls(df)

column_names = []
for column in df.columns:
    name = column.replace(" ", "_")
    column_names.append(name)
df.columns = column_names


In [3]:
df_train, df_test = utils.dividir_df_testeo(df, test_size=0.15)

In [4]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

In [5]:
categoricas = ['tipodepropiedad', 'provincia', 'zona', 'intervalo_metros_cubiertos', 'intervalo_metros_totales', 'escualas_centros_cercanos']
categoricas_ohe = []

for i in categoricas:
    df_test_f, nuevas_cols = features.columna_a_ohe(df_test_f, i, df_aux=df_train_f, devolver_cols=True)
    df_train_f = features.columna_a_ohe(df_train_f, i, df_aux=df_test_f)
    categoricas_ohe += nuevas_cols

In [6]:
features = list(df_train_f.columns)

features_drop = ['id', 'titulo', 'descripcion', 'direccion', 'intervalo_metros_cubiertos', 
                 'intervalo_metros_totales', 'provincia', 'escualas_centros_cercanos', 'tipodepropiedad', 
                 'ciudad', 'idzona', 'promedio_precio_hbg_tipo_propiedad_provincia_gen', 'zona', 'idzona']

features = [f for f in features if f not in features_drop]

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features)

In [7]:
x_train.loc[:, x_train.isnull().any()]

""
36242
203272
201436
114481
65169
...
73349
109259
50057
5192


In [ ]:
zona = x_train.drop(['titulo', 'descripcion','direccion', 'zona', 'intervalo_metros_cubiertos', 'intervalo_metros_totales', 'provincia', 'escualas_centros_cercanos', 'tipodepropiedad', 'ciudad', 'habitaciones', 'garages', 'banos', 'antiguedad', 'idzona', 'lat', 'lng', 'promedio_precio_hbg_tipo_propiedad_provincia_gen'], axis=1)
df_encoding = pd.merge(zona, categoricos, on= 'id', how= 'left')

zona_test = x_test.drop(['titulo', 'descripcion','direccion', 'zona', 'intervalo_metros_cubiertos', 'intervalo_metros_totales', 'provincia', 'escualas_centros_cercanos', 'tipodepropiedad', 'ciudad', 'habitaciones', 'garages', 'banos', 'antiguedad', 'idzona', 'lat', 'lng', 'promedio_precio_hbg_tipo_propiedad_provincia_gen'], axis=1)
df_encoding_test = pd.merge(zona_test, categoricos_test, on= 'id', how= 'left')
df_encoding_test = df_encoding_test.drop(['tipodepropiedad_Garage', 'tipodepropiedad_Hospedaje'],axis =1)


In [7]:
clf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0, n_jobs=4, verbose=1)
clf.fit(x_train,y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.4min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                      oob_score=False, random_state=0, verbose=1,
                      warm_start=False)

In [1]:
print(categoricos.shape)
print(categoricos_test.shape)
print(df_encoding.shape)
print(df_encoding_test.shape)

set(df_encoding_test.columns)  - set(df_encoding.columns)

NameError: name 'categoricos' is not defined

In [ ]:
clf.predict(df_encoding_test)

In [ ]:
clf.feature_importances_

In [8]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)
        display(df.shape)
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(x_train.columns, clf.feature_importances_):
    feats[feature] = importance #add the name/value pair 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
full_display(importances.sort_values('Gini-importance', ascending=False))
# importances.sort_values(by='Gini-importance').nlargest(40, 'Gini-importance').plot(kind='barh', rot=0, figsize=(12, 12))

,Gini-importance
promedio_precio_hbg_tipo_propiedad_provincia,5.371382e-01
promedio_id_zona,1.195847e-01
promedio_id_zona_log,1.188250e-01
puntaje,3.741368e-02
varianza_id_zona,2.860231e-02
tam_ambientes,1.921727e-02
metros_cubiertos_normalizados,1.762405e-02
metroscubiertos,1.485811e-02
metroscubiertos_log,1.286173e-02
metrostotales_log,1.116415e-02


(139, 1)

In [ ]:
cubiertos = importances.transpose()
cubiertos = cubiertos[['metroscubiertos', 'intervalo_metros_cubiertos_Grupo1', 'intervalo_metros_cubiertos_Grupo2', 'intervalo_metros_cubiertos_Grupo3', 'intervalo_metros_cubiertos_Grupo4', 'intervalo_metros_cubiertos_Grupo5']]
cubiertos.transpose()

In [ ]:
totales = importances.transpose()
totales = totales[['metrostotales', 'intervalo_metros_totales_Grupo1', 'intervalo_metros_totales_Grupo2', 'intervalo_metros_totales_Grupo3', 'intervalo_metros_totales_Grupo4', 'intervalo_metros_totales_Grupo5']]
totales.transpose()